<a href="https://colab.research.google.com/github/vsingh9076/Computer_Vision/blob/main/Basic_CNN/MNIST_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR,OneCycleLR

In [ ]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()

    self.conv1 = nn.Sequential(
        nn.Conv2d(1, 8, 3),
        nn.ReLU(),
        nn.BatchNorm2d(8),
        nn.Dropout(0.1)
        )

    self.conv2 = nn.Sequential(
        nn.Conv2d(8, 16, 3),
        nn.ReLU(),
        nn.BatchNorm2d(16),
        nn.Dropout(0.1)
        )

    self.conv3 = nn.Sequential(
        nn.Conv2d(16, 32, 3),
        nn.ReLU(),
        nn.BatchNorm2d(32),
        nn.Dropout(0.1)
        )

    self.conv3_1 = nn.Sequential(
        nn.Conv2d(16, 32, 3)
    )

    self.conv1_1 = nn.Sequential(
        nn.Conv2d(32, 8, 1),
        nn.ReLU(),
        nn.BatchNorm2d(8),
        nn.Dropout(0.1)
       )

    self.maxpool = nn.MaxPool2d(2, 2)

    self.fc = nn.Linear(in_features=5*5*32, out_features=10)

    self.avg_pool = nn.Sequential(
            nn.AvgPool2d(5, stride=1, padding=0)
        )

  def forward(self, x):
    x = self.conv1(x)     # (28, 28, 1) -->  (26, 26, 8)   ; RF : 3x3
    x = self.conv2(x)     # (26, 26, 8) -->  (24, 24, 16)  ; RF : 5x5
    x = self.conv3(x)     # (24, 24, 16) --> (22, 22, 32)  ; RF : 7x7
    x = self.conv1_1(x)   # (22, 22, 32) --> (22, 22, 8)   ; RF : 7x7  # Transition Block (1x1)
    x = self.conv2(x)     # (22, 22, 8) --> (20, 20, 16)   ; RF : 9x9
    x = self.conv3(x)     # (20, 20, 16) --> (18, 18, 32)  ; RF : 11x11
    x = self.maxpool(x)   # (18, 18, 32) --> (9, 9, 32)    ; RF : 22x22 # MaxPool
    x = self.conv1_1(x)   # (9, 9, 32) --> (9, 9, 8)       ; RF : 22x22 # Transition Block (1x1)
    x = self.conv2(x)     # (9, 9, 8) --> (7, 7, 16)       ; RF : 24x24
    x = self.conv3_1(x)   # (7, 7, 16) --> (5, 5, 32)      ; RF : 26x26
    x = x.reshape(-1, 32 * 5 * 5)
    x = self.fc(x)
    x = x.view(x.size(0), -1)
    x = F.log_softmax(x, dim=1)

    return x


# !pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
# model = Model().to(device)
summary(model, input_size=(1, 28, 28))



----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              80
              ReLU-2            [-1, 8, 26, 26]               0
       BatchNorm2d-3            [-1, 8, 26, 26]              16
           Dropout-4            [-1, 8, 26, 26]               0
            Conv2d-5           [-1, 16, 24, 24]           1,168
              ReLU-6           [-1, 16, 24, 24]               0
       BatchNorm2d-7           [-1, 16, 24, 24]              32
           Dropout-8           [-1, 16, 24, 24]               0
            Conv2d-9           [-1, 32, 22, 22]           4,640
             ReLU-10           [-1, 32, 22, 22]               0
      BatchNorm2d-11           [-1, 32, 22, 22]              64
          Dropout-12           [-1, 32, 22, 22]               0
           Conv2d-13            [-1, 8, 22, 22]             264
             ReLU-14            [-1, 8,

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 18,930 trainable parameters


In [ ]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [ ]:
from tqdm import tqdm

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

for epoch in range(1, 20):
  print('Epoch :', epoch)
  train(model, device, train_loader, optimizer, epoch)
  test(model, device, test_loader)

Epoch : 1


loss=0.027234604582190514 batch_id=468: 100%|██████████| 469/469 [00:18<00:00, 25.17it/s]



Test set: Average loss: 0.0768, Accuracy: 9756/10000 (97.56%)

Epoch : 2


loss=0.034358128905296326 batch_id=468: 100%|██████████| 469/469 [00:15<00:00, 30.09it/s]



Test set: Average loss: 0.0623, Accuracy: 9799/10000 (97.99%)

Epoch : 3


...


Test set: Average loss: 0.0365, Accuracy: 9878/10000 (98.78%)

Epoch : 18


loss=0.007671449799090624 batch_id=468: 100%|██████████| 469/469 [00:15<00:00, 30.18it/s]



Test set: Average loss: 0.0442, Accuracy: 9856/10000 (98.56%)

Epoch : 19


loss=0.047103866934776306 batch_id=468: 100%|██████████| 469/469 [00:15<00:00, 30.02it/s]



Test set: Average loss: 0.0382, Accuracy: 9883/10000 (98.83%)

